# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [1]:
!wget 'https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2024/homeworks/hw08_boosting_clustering/ds_salaries.csv'

--2024-06-18 16:58:39--  https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2024/homeworks/hw08_boosting_clustering/ds_salaries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210076 (205K) [text/plain]
Saving to: ‘ds_salaries.csv’

ds_salaries.csv     100%[===================>] 205.15K  --.-KB/s    in 0.03s   

2024-06-18 16:58:39 (5.89 MB/s) - ‘ds_salaries.csv’ saved [210076/210076]



In [2]:
import pandas as pd

df = pd.read_csv("ds_salaries.csv")
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [3]:
y=df['salary_in_usd']
X = df.drop(['salary_in_usd','salary_currency', 'salary'], axis=1)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2004)


In [5]:
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size=0.5, random_state=2004)


## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

np.random.seed(42)

categorical_features = ['experience_level','employment_type','job_title','employee_residence','company_location','company_size']

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

X_train_encoded = encoder.fit_transform(X_train[categorical_features])

X_val_encoded = encoder.transform(X_val[categorical_features])
X_test_encoded = encoder.transform(X_test[categorical_features])




X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out())
X_val_encoded = pd.DataFrame(X_val_encoded, columns=encoder.get_feature_names_out())
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out())


X_train_numbers=X_train.drop(categorical_features, axis=1)
X_train_encoded.reset_index(drop=True, inplace=True)
X_train_numbers.reset_index(drop=True, inplace=True)
X_train_final = pd.concat([X_train_encoded,X_train_numbers], axis=1)


X_val_numbers=X_val.drop(categorical_features, axis=1)
X_val_encoded.reset_index(drop=True, inplace=True)
X_val_numbers.reset_index(drop=True, inplace=True)
X_val_final = pd.concat([X_val_encoded,X_val_numbers], axis=1)



X_test_numbers=X_test.drop(categorical_features, axis=1)
X_test_encoded.reset_index(drop=True, inplace=True)
X_test_numbers.reset_index(drop=True, inplace=True)
X_test_final = pd.concat([X_test_encoded,X_test_numbers], axis=1)


model = LinearRegression()
model.fit(X_train_final, y_train)
y_pred_test = model.predict(X_test_final)

MAPE = mean_absolute_percentage_error(y_test, y_pred_test)
RMSE = mean_squared_error(y_test, y_pred_test) ** 0.5

print('MAPE: ', MAPE )
print('RMSE: ', RMSE )

MAPE:  42542580.303004846
RMSE:  15505427289034.615


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [7]:
pip install xgboost

In [8]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

np.random.seed(42)

params = {
    'max_depth' : [4,5,6,7,8,9],
    'learning_rate' : [0.01,0.05,0.1],
    'n_estimators' : [25,50,75,100,150],
    'gamma' : [0,0.1,0.2],
    'random_state': [42]
}
model = XGBRegressor()
grid_search = GridSearchCV(estimator = model, param_grid=params, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_val_final, y_val)


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'gamma': [0, 0.1, 0.2],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [4, 5, 6, 7, 8, 9],
                         'n_estimators': [25, 50, 75, 100, 150],
                         'random_state': [42]},
             scoring='neg_mean_squared_error')

In [9]:
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 75, 'random_state': 42}
Best score: -2611165554.8101


In [10]:
model = XGBRegressor(max_depth=4, learning_rate=0.1, n_estimators=75, gamma=0,random_state=42)
%time model.fit(X_train_final,y_train)
%time y_pred = model.predict(X_test_final)

MAPE = mean_absolute_percentage_error(y_test, y_pred)
RMSE = mean_squared_error(y_test, y_pred) ** 0.5

print('MAPE: ', MAPE )
print('RMSE: ', RMSE )

CPU times: user 466 ms, sys: 5.93 ms, total: 472 ms
Wall time: 265 ms
CPU times: user 44.8 ms, sys: 994 µs, total: 45.8 ms
Wall time: 30.8 ms
MAPE:  0.4285939439921434
RMSE:  52318.747869995255


## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [11]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.1 MB/s eta 0:00:00


In [12]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
params = {
    'depth' :[4,5,6,7,8,9] ,
    'learning_rate' : [0.01,0.05,0.1] ,
    'iterations' : [25,50,75,100,150],
    'random_state': [42]

}

model = CatBoostRegressor()
grid_search = GridSearchCV(estimator = model, param_grid=params, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_val_final, y_val)


Выходные данные были обрезаны до нескольких последних строк (5000).
25:	learn: 61061.4107449	total: 102ms	remaining: 193ms
26:	learn: 60942.6713380	total: 103ms	remaining: 184ms
27:	learn: 60835.3412734	total: 104ms	remaining: 174ms
28:	learn: 60713.8046937	total: 107ms	remaining: 169ms
29:	learn: 60590.0570767	total: 108ms	remaining: 163ms
30:	learn: 60456.9811786	total: 111ms	remaining: 158ms
31:	learn: 60343.5604472	total: 114ms	remaining: 154ms
32:	learn: 60229.5680802	total: 116ms	remaining: 148ms
33:	learn: 60100.5427003	total: 121ms	remaining: 146ms
34:	learn: 60001.8960048	total: 128ms	remaining: 146ms
35:	learn: 59868.2015286	total: 137ms	remaining: 148ms
36:	learn: 59766.4326328	total: 140ms	remaining: 144ms
37:	learn: 59663.2201381	total: 147ms	remaining: 143ms
38:	learn: 59532.6646232	total: 154ms	remaining: 142ms
39:	learn: 59432.9214762	total: 158ms	remaining: 138ms
40:	learn: 59329.0358017	total: 159ms	remaining: 132ms
41:	learn: 59218.2964749	total: 159ms	remaining: 125

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7c3aa2061fc0>,
             param_grid={'depth': [4, 5, 6, 7, 8, 9],
                         'iterations': [25, 50, 75, 100, 150],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'random_state': [42]},
             scoring='neg_mean_squared_error')

In [13]:
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'depth': 5, 'iterations': 150, 'learning_rate': 0.1, 'random_state': 42}
Best score: -2546885004.6749434


In [14]:
model = CatBoostRegressor(depth=5, iterations=150,learning_rate=0.1,random_seed=42)
%time model.fit(X_train_final,y_train)
%time y_pred = model.predict(X_test_final)

MAPE = mean_absolute_percentage_error(y_test, y_pred)
RMSE = mean_squared_error(y_test, y_pred) ** 0.5

print('MAPE: ', MAPE )
print('RMSE: ', RMSE )

0:	learn: 60100.6056884	total: 12ms	remaining: 1.79s
1:	learn: 58306.6259058	total: 14.5ms	remaining: 1.07s
2:	learn: 57056.7358341	total: 18.2ms	remaining: 890ms
3:	learn: 55803.0293057	total: 21.5ms	remaining: 785ms
4:	learn: 54756.8606595	total: 25ms	remaining: 725ms
5:	learn: 53823.9716682	total: 33.2ms	remaining: 797ms
6:	learn: 53056.7764197	total: 36.3ms	remaining: 741ms
7:	learn: 52360.3445730	total: 39.4ms	remaining: 700ms
8:	learn: 51791.1963496	total: 46.8ms	remaining: 732ms
9:	learn: 51258.0525857	total: 49.7ms	remaining: 696ms
10:	learn: 50890.7098712	total: 52.5ms	remaining: 664ms
11:	learn: 50532.4024565	total: 55.3ms	remaining: 636ms
12:	learn: 50307.7231075	total: 63ms	remaining: 663ms
13:	learn: 50039.9236256	total: 69.3ms	remaining: 673ms
14:	learn: 49799.1178541	total: 73.3ms	remaining: 659ms
15:	learn: 49612.1176823	total: 77.9ms	remaining: 653ms
16:	learn: 49401.2101021	total: 81.6ms	remaining: 638ms
17:	learn: 49223.1015708	total: 84.6ms	remaining: 620ms
18:	lear

Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [15]:
from catboost import Pool
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

categorical_features = ['experience_level','employment_type','job_title','employee_residence','company_location','company_size']

val_pool = Pool(data=X_val, label=y_val, cat_features=categorical_features)

params = {
    'depth' :[4,5,6,7,8,9] ,
    'learning_rate' : [0.01,0.05,0.1] ,
    'iterations' : [25,50,75,100,150]

}


model = CatBoostRegressor()

grid_search_result = model.grid_search(params,
                                       X=val_pool,
                                       cv=5,
                                       partition_random_seed=42,
                                       )


Выходные данные были обрезаны до нескольких последних строк (5000).
26:	learn: 69066.8860875	test: 65890.9568954	best: 65890.9568954 (26)	total: 64.9ms	remaining: 175ms
27:	learn: 68107.1105377	test: 65021.1168311	best: 65021.1168311 (27)	total: 65.3ms	remaining: 168ms
28:	learn: 67185.6460577	test: 64246.4348040	best: 64246.4348040 (28)	total: 65.9ms	remaining: 161ms
29:	learn: 66366.4783556	test: 63526.1665632	best: 63526.1665632 (29)	total: 67.6ms	remaining: 158ms
30:	learn: 65432.5918725	test: 62485.1155564	best: 62485.1155564 (30)	total: 69.8ms	remaining: 155ms
31:	learn: 64737.6170777	test: 61877.9122597	best: 61877.9122597 (31)	total: 70.7ms	remaining: 150ms
32:	learn: 64024.6573952	test: 61363.9987037	best: 61363.9987037 (32)	total: 71.7ms	remaining: 146ms
33:	learn: 63274.3161433	test: 60773.2030058	best: 60773.2030058 (33)	total: 73.9ms	remaining: 144ms
34:	learn: 62579.6885556	test: 60234.3378888	best: 60234.3378888 (34)	total: 76.1ms	remaining: 141ms
35:	learn: 61734.677528

In [16]:
grid_search_result['params']

{'depth': 4, 'learning_rate': 0.1, 'iterations': 100}

In [18]:
categorical_features = ['experience_level','employment_type','job_title','employee_residence','company_location','company_size']
model = CatBoostRegressor(depth=4, iterations=100,learning_rate=0.1,random_seed=42)
%time model.fit(X_train,y_train,cat_features=categorical_features)
%time y_pred = model.predict(X_test)

MAPE = mean_absolute_percentage_error(y_test, y_pred)
RMSE = mean_squared_error(y_test, y_pred) ** 0.5

print('MAPE: ', MAPE )
print('RMSE: ', RMSE )

0:	learn: 61019.4745995	total: 7.02ms	remaining: 695ms
1:	learn: 60065.4474761	total: 11ms	remaining: 540ms
2:	learn: 58628.0998768	total: 23.7ms	remaining: 765ms
3:	learn: 57377.7110731	total: 29ms	remaining: 696ms
4:	learn: 56343.3374334	total: 30.7ms	remaining: 584ms
5:	learn: 55455.5366426	total: 34.6ms	remaining: 542ms
6:	learn: 54772.1471689	total: 38.2ms	remaining: 507ms
7:	learn: 54161.2898563	total: 43.2ms	remaining: 497ms
8:	learn: 53595.7797056	total: 48.1ms	remaining: 486ms
9:	learn: 53193.4592837	total: 52.8ms	remaining: 475ms
10:	learn: 52784.5260850	total: 63.7ms	remaining: 515ms
11:	learn: 52500.7144434	total: 69.5ms	remaining: 509ms
12:	learn: 51967.6422230	total: 86.7ms	remaining: 580ms
13:	learn: 51519.6250402	total: 90.2ms	remaining: 554ms
14:	learn: 51145.2056110	total: 95.3ms	remaining: 540ms
15:	learn: 50810.9463399	total: 107ms	remaining: 561ms
16:	learn: 50509.6079122	total: 112ms	remaining: 545ms
17:	learn: 50334.3182228	total: 116ms	remaining: 527ms
18:	learn

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [19]:
pip install lightgbm

In [20]:
from lightgbm import LGBMRegressor


params = {
    'max_depth' : [4,5,6,7,8,9] ,
    'learning_rate' : [0.01,0.05,0.1],
    'n_estimators' : [25,50,75,100,150],
    'random_state': [42]


}

model = LGBMRegressor()
grid_search = GridSearchCV(estimator = model, param_grid=params, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_val_final, y_val)

Выходные данные были обрезаны до нескольких последних строк (5000).
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [4, 5, 6, 7, 8, 9],
                         'n_estimators': [25, 50, 75, 100, 150],
                         'random_state': [42]},
             scoring='neg_mean_squared_error')

In [21]:
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 75, 'random_state': 42}
Best score: -2679039605.9188833


In [22]:
model = LGBMRegressor(learning_rate=0.1,max_depth=4,n_estimatros=75,random_state=42)
%time model.fit(X_train_final,y_train)
%time y_pred = model.predict(X_test_final)

MAPE = mean_absolute_percentage_error(y_test, y_pred)
RMSE = mean_squared_error(y_test, y_pred) ** 0.5

print('MAPE: ', MAPE )
print('RMSE: ', RMSE )

[LightGBM] [Warning] Unknown parameter: n_estimatros
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: n_estimatros
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000902 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 76
[LightGBM] [Info] Number of data points in the train set: 3004, number of used features: 36
[LightGBM] [Info] Start training from score 137616.321238
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

In [25]:
# XGBOOST
# CPU times: user 466 ms, sys: 5.93 ms, total: 472 ms
# Wall time: 265 ms
# CPU times: user 44.8 ms, sys: 994 µs, total: 45.8 ms
# Wall time: 30.8 ms
# MAPE:  0.4285939439921434
# RMSE:  52318.747869995255

# CATBOOST(без пул)
# CPU times: user 566 ms, sys: 127 ms, total: 693 ms
# Wall time: 1.21 s
# CPU times: user 26.3 ms, sys: 952 µs, total: 27.2 ms
# Wall time: 54.6 ms
# MAPE:  0.4207307124488648
# RMSE:  52156.27706700808

# CATBOOST(пул)
# CPU times: user 380 ms, sys: 42.2 ms, total: 423 ms
# Wall time: 437 ms
# CPU times: user 3.48 ms, sys: 8 µs, total: 3.49 ms
# Wall time: 2.25 ms
# MAPE:  0.4428781522233773
# RMSE:  52974.53681764346

# lightGBM
# CPU times: user 124 ms, sys: 21.9 ms, total: 146 ms
# Wall time: 187 ms
# CPU times: user 5.53 ms, sys: 1.98 ms, total: 7.5 ms
# Wall time: 7.57 ms
# MAPE:  0.4344386065069441
# RMSE:  52584.4042364759

# По качеству лучшая модель бустинга в данном случае это CATBOOST с самостоятельным отбором категориальных призанков (без пула), так как там и MAPE, и RMSE самые маленькие.
# Худшая же по качеству модель это CATBOOST с пуллом(имеет худшие показатели как по MAPE, так и по RMSE)
# Скорость обучения самая большая у lightGBM(146 ms), самым же медленным при обучении оказался CATBOOST без пул (у CATBOOST с пул и XGBOOST время примерно одинаковое)
# Скорость предсказания самая быстрая у CATBOOST(пул), а самая медленная у  XGBOOST(целых в 13 раз)
# Стоит отметить, что у всех моделей есть параметр learning_rate (у всех моделей он равен  0.1), n_estimators/ iterations (принимает разные значения : 75, 100, 150), а также
# max_depth/depth(во всех моделях 4, а в  СATBOOST без пула - 5)





# БОНУСНУЮ ЧАСТЬ Я НЕ ДЕЛАЛ

**Ответ:** # -- YOUR ANSWER HERE --

# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [ ]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [ ]:
# -- YOUR CODE HERE --

Выкиньте строку под названием `user`.

In [ ]:
# -- YOUR CODE HERE --

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [ ]:
# -- YOUR CODE HERE --
ratings.sample()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
ben harper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [ ]:
from sklearn.cluster import KMeans

# -- YOUR CODE HERE --

Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [ ]:
# -- YOUR CODE HERE --

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --